In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
from pynq.lib import Pmod_PWM

pwm = Pmod_PWM(base.PMODA,0)

In [3]:
import time
# Generate a 10 us clocks with 50% duty cycle
period=10000
# duty=25
duty=10
pwm.generate(period,duty)

In [65]:
base?

In [4]:
import time
import cv2
from matplotlib import pyplot as plt  
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random
from pynq import allocate
import numpy as np

In [5]:
in_arr = allocate(shape=(600,800,3),dtype=np.uint8)
out_arr = allocate(shape=(600,800,3),dtype=np.uint8)

img = cv2.imread("test.jpg")

np_out = np.zeros((600,800,3), dtype=np.uint8)

In [6]:
ip = base.ImgProcess_Top_0

# dma data transfer
dma = base.axi_dma_0


In [7]:
np.copyto(in_arr,img)

In [9]:
dma.sendchannel.transfer(in_arr)
dma.recvchannel.transfer(out_arr)

In [10]:
ip.write(0x00, 0x1)
isready = ip.read(0x00)
while (isready == 1):
    isready = ip.read(0x00)
# finally wait for ending
dma.sendchannel.wait()
dma.recvchannel.wait()

In [20]:
out_arr

PynqBuffer([[[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0],
             ...,
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]],

            [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0],
             ...,
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]],

            [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0],
             ...,
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]],

            ...,

            [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0],
             ...,
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]],

            [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0],
             ...,
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]],

            [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0],
             ...,
             [0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]]]

In [11]:
num = ip.read(0x18)
sum = ip.read(0x10)

In [12]:
print(sum)
print(num)

51647840
79501


In [13]:
51647840/79501

649.6501930793323

In [46]:
pwm.stop()

In [95]:
def leftshift(primitive,angle):
    primitive = primitive + 50*angle
    if(primitive > 10500):
        primitive = 10500
    pwm.generate(primitive,duty)
    return primitive
    
def rightshift(primitive,angle):
    primitive = primitive - 50*angle
    if(primitive < 9500):
        primitive = 9500
    pwm.generate(primitive,duty)
    return primitive

In [9]:
angle = 10000

In [93]:
angle = rightshift(angle,4)
angle

10000

In [10]:
import time
import cv2
from matplotlib import pyplot as plt  
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random
from pynq import allocate
import numpy as np

In [11]:
in_arr = allocate(shape=(600,800,3),dtype=np.uint8)
out_arr = allocate(shape=(600,800,3),dtype=np.uint8)

img = cv2.imread("test.jpg")

np_out = np.zeros((600,800,3), dtype=np.uint8)

In [12]:
ip = base.ImgProcess_Top_0
# dma data transfer
dma = base.axi_dma_0

In [13]:
import time
import cv2                                   


from matplotlib import pyplot as plt  
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random
from pynq import allocate
import numpy as np
import socket, time

In [44]:
#open camera
frame_in_w = 1280
frame_in_h = 720

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)#set picture width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)#set hight
cap.open(0)

True

In [122]:
#open camera 
frame_in_w = 1280 
frame_in_h = 720
cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)#set picture width 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)#set hight 
cap.open(0)
while True:
    for i in range(1,5):
        ret, np_frame = cap.read()
        np_frame = cv2.resize(np_frame,(800,600))
        np.copyto(in_arr,np_frame)
        dma.sendchannel.transfer(in_arr)
        dma.recvchannel.transfer(out_arr)
        ip.write(0x00, 0x1)
        isready = ip.read(0x00)
        while (isready == 1):
            isready = ip.read(0x00)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        sum = ip.read(0x10)
        num = ip.read(0x18)
        out.write(np_frame)
    pos = sum/(num+1)
    print(pos)
    if(pos > 550):
        angle = rightshift(angle,2)
    if(pos < 250):
        angle = leftshift(angle,2)

399.76167883566507
394.7972135515798
470.06547568813653
536.3638938198385
542.6952959753589
552.1277841036065
556.91881869951
537.6918677629478
601.8186324053785
624.0884737938642
654.8767079362252
662.1860657397408
356.2628410787459
264.7016761394852
231.0228367038107
238.71713643168695
246.98822220356064
279.05092697213206
271.258107113225
253.1104726009762
267.0793783510399
298.8906254017722
233.97208850888117
224.43731260827823
242.2308291613164
311.8286668501001
337.0134678147556
494.30300967937086
508.43496917202594
546.0770728730339
541.858441098709
500.6574392952509
505.5087378915765
480.85913806397343
516.2428591582923
527.4207936443185
527.541498691276
533.7286726341745
531.706890910626
545.0953805711555
555.1391019201793
551.9060719073426
518.925792876406
423.7726840840732
440.76407267536945
87.41950491268271
95.20868663679786
97.00145926817702
114.4286067193676
140.38156028368795
168.4708504618013
202.830789543174
245.5444877415165
238.5600239279985
544.8327804745182
251.09

KeyboardInterrupt: 

In [84]:
angle = 10000
pwm.generate(angle,duty)

In [ ]:
np.copyto(np_out,out_arr)
from matplotlib import pyplot as plt  
plt.imshow(np_out[:,:,[2,1,0]])

In [21]:
angle

11400

In [78]:
while True:
    ret, np_frame = cap.read()
    np_frame = cv2.resize(np_frame,(800,600))
    np.copyto(in_arr,np_frame)
    dma.sendchannel.transfer(in_arr)
    dma.recvchannel.transfer(out_arr)
    ip.write(0x00, 0x1)
    isready = ip.read(0x00)
    while (isready == 1):
        isready = ip.read(0x00)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    sum = ip.read(0x10)
    num = ip.read(0x18)
    pos = sum/(num+1)
    print(pos)

139.27894612939932
139.06307595780558
137.35008651879818
137.74160328596977
137.59982347749337
138.36404736275566
136.66148824038385
134.42808239507687
134.22903788239023
137.04991818307278
136.3375659500748
138.86044591246903
137.16817912803313


KeyboardInterrupt: 

In [120]:
cap.release()

#open camera
frame_in_w = 1280
frame_in_h = 720

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)#set picture width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)#set hight
cap.open(0)

In [119]:
angle = 10000
pwm.generate(angle,duty)

In [77]:
#open camera 
frame_in_w = 1280 
frame_in_h = 720
cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)#set picture width 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)#set hight 
cap.open(0)

True

In [121]:
cap2 = cv2.VideoCapture(1) 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('testwrite.mp4',fourcc, 10.0, (800,600),True)

In [123]:
cap2.release()
out.release()

In [101]:
angle

10500